In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import mmcv
import importlib
import torch
import torchvision.transforms as transforms
from mmcv import Config
from mmcls.datasets.pipelines import Compose
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from PIL import Image
import heapq
from mmseg.apis import inference_segmentor, init_segmentor
from mmseg.core.evaluation import get_palette
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

from scripts import generate_statistics_single
from scripts import visualization_seg_masks
from scripts import generate_statistics
from scripts import generate_cams
from scripts import utils
from scripts.utils.pipeline import get_pipeline_torchvision
from scripts import generate_segs

In [2]:
configSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/ocrnet_hr48_512x512_carparts_noflip.py'
checkpointSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/latest.pth'
imgRootTrain = '../data/CompCars_sv_original_split/train'
imgRootVal = '../data/CompCars_sv_original_split/val'
configCAM = '../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py'
checkpointCAM = '../CAMModels/resnet/compCars_Original/latest.pth'
annfileTrain = '../annfileTrain.txt'
annfileVal = '../annfileVal.txt'
annfileValGt = '../annfileValGt.txt'
imgNameTrain='Acura_Acura_ILX_2a6cce617fc27d.jpg'
imgNameVal = 'Acura_Acura_ILX_3c11ef9e42931b.jpg'
imgPathTrain=os.path.join(imgRootTrain, imgNameTrain)
imgPathVal=os.path.join(imgRootVal, imgNameVal)

paletteName='Comp_Original_Ocrnet_Carparts_Noflip'

In [3]:
imgRootValOriginal = '../data/CompCars_sv_original_split/val'
imgRootValColor = '../data/CompCars_sv_color_split/val'
imgRootValWeb = '../data/CompCars_web_original_split/val'

In [5]:
generate_cams.main([imgRootValColor, configCAM, checkpointCAM, '-s', '../preGenData/ColorvalAllCAMs'])
generate_cams.main([imgRootValColor, configCAM, checkpointCAM, '-s', '../preGenData/ColorvalAllCAMsGrad++', '--method', 'gradcam++'])
generate_cams.main([imgRootValColor, configCAM, checkpointCAM, '-s', '../preGenData/ColorvalAllCAMsXGrad', '--method', 'xgradcam'])
generate_cams.main([imgRootValColor, configCAM, checkpointCAM, '-s', '../preGenData/ColorvalAllCAMsEigen', '--method', 'eigencam'])
generate_cams.main([imgRootValColor, configCAM, checkpointCAM, '-s', '../preGenData/ColorvalAllCAMsEigenGrad', '--method', 'eigengradcam'])
generate_cams.main([imgRootValColor, configCAM, checkpointCAM, '-s', '../preGenData/ColorvalAllCAMsLayer', '--method', 'layercam'])
generate_cams.main([imgRootValColor, configCAM, checkpointCAM, '-s', '../preGenData/ColorvalAllCAMsGPU', '--device', 'cuda'])

Method for CAM generation: GradCAM, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13379/13379, 4.1 task/s, elapsed: 3276s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData\ColorvalAllCAMs.npz
Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13379/13379, 4.7 task/s, elapsed: 2836s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData\ColorvalAllCAMsGrad++.npz
Method for CAM generation: xgradcam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13379/13379, 4.7 task/s, elapsed: 2844s, ETA:     0sSave Split file for Cams
Save generate